In [ ]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import sklearn
from sklearn.preprocessing import StandardScaler

import pandas as pd

In [ ]:
exact_data = pd.read_excel("../UNT-Thermography-24/fused_silica_data/fused_silica_exact_train.xlsx")

exact_df = pd.DataFrame(exact_data)

X = exact_df[exact_df.columns[11:]]

Y = exact_df[exact_df.columns[0:11]]

print(f"The original data set has {exact_df.shape[0]} rows, and {exact_df.shape[1]} columns. "
      f"We want to predict {target.shape[1]} output variables from {features.shape[1]} features")

exact_df.head()

In [ ]:
#Normalize features
scaler = sklearn.preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
#Goal: Using keras to implement a feed-forward neural network to predict temperatures of each layer given wavelength spectrum
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled_df, Y, train_size = .8, random_state = 1337)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, train_size=0.8, random_state = 1337)

print(
    f"Using {len(Y_train)} samples for training, "
    f"and {len(Y_test)} for testing "
    f"from {len(X_test) + len(X_train)} total samples."
)

In [ ]:
# Define the input shape
input_layer = keras.Input(shape=(X_train.shape[1],))

#Define the model (add more layers? increase nodes?)
exact_model = keras.Sequential([input_layer, #input_layer --> informs input shape
    layers.Dense(256, activation="relu"), # first hidden layer
    layers.Dense(128, activation="relu"), # second hidden layer
    # layers.Dense(20, activation="relu"), # third hidden layer
    # layers.Dense(20, activation="relu"), # fourth hidden layer
    layers.Dense(11)  # Output layer with 11 neurons for each temperature target
])

# Compile the model
# Instantiate optimizer with adjustable learning rate
# observed: sweet spot .02 > x > .001
Adam1 = Adam(learning_rate = 0.009)

# Leverage both MSE and MAE (experimental)
# test different optimizers: SGD (*stochastic gradient descent: better for large data sets)
exact_model.compile(loss="mse", optimizer=Adam1, metrics=["mae"])  # Loss: MSE, Optimizer: Adam w/ MAE

# Train the model
# Thoughts: Overfit then implement regularization
# play around with batch size: batch size for larger datasets more likely to improve its performance
# Dima went up to 60 epochs for GaN_exact: later 600 -> imrpovement plateaus around 120 epochs
# plot loss over epochs
history = exact_model.fit(X_train, Y_train, epochs=15, validation_data = [X_val, Y_val])

#View Training Error
print(f"train_MSE: {history.history["loss"]}")
print(f"train_MAE: {history.history.get("mae", None)}")

# Evaluate the model on the validation set (separate call)
validation_loss, validation_accuracy = exact_model.evaluate(X_val, Y_val)

print(f"val_MSE: {validation_loss}, val_MAE: {validation_accuracy}")

In [ ]:
# Evaluate the model on unseen data
loss, mae = exact_model.evaluate(X_test, Y_test)
print("test_MSE:", loss)
print("test_MAE:", mae)

In [ ]:
#TODO:
#plot predictions against actual (scatter plot to see if similar)
#m nodes by n layers: want m*n to be less than number of data points
#per layer residual
#avg (above) to get overall residual
#plot error over epochs
#increase layers: implement loop
#change activation function to hyperbolic tangent: (theoretical difference: ReLu 0 and 1, tangent maintains positive and negative (if normalized data is positive and negative, it is possible tanegent would work better (consider what might happen if all data is positive)))
# 20 layers 250 nodes, batch size 15, epochs 220
#learning rate vs momenta/momento?? may reduce epochs